In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import copy
from math import sqrt,ceil
from scipy.stats import linregress
import libraries as lib

In [6]:
directory = "../../../OneDrive - HvA/Logfiles/H2A/2022/20220628-20220701_race_nogaro/0023 - 20220701T141014 1h8m8s 13.336km Nogaro/"
file = "1.csv"

In [7]:
data_file = lib.DataInladen(f'{directory}{file}')

In [8]:
data_file

,Datalogger port,"Dataloggertijd, in s",Format header (>08|03),NMEA type($GPRMC),Tijd,Status,Latitude,Noord of Zuid (N/S),Longitude,Oost of West (E/W),...,Mode indicator (A),Data in balancers,DegreesTrue,T,DegreesMagnetic,M,Snelheid tenopzichte van het water in Knots,N,Snelheid tenopzichte van het water in Kilometers,K
0,1,70.8626,08|03,$GPRMC,120813.0,V,0.0000,N,0.0000,E,...,N,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,71.8578,08|03,$GPRMC,120814.0,V,0.0000,N,0.0000,E,...,N,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,72.8577,08|03,$GPRMC,120815.0,V,0.0000,N,0.0000,E,...,N,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,73.8609,08|03,$GPRMC,120816.0,V,0.0000,N,0.0000,E,...,N,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,74.8585,08|03,$GPRMC,120817.0,V,0.0000,N,0.0000,E,...,N,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4200,1,4276.0450,08|03,$GPRMC,131818.0,A,4346.3114,N,2.5422,W,...,A,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4201,1,4277.0633,08|03,$GPRMC,131819.0,A,4346.3114,N,2.5422,W,...,A,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4202,1,4278.0288,08|03,$GPRMC,131820.0,A,4346.3114,N,2.5422,W,...,A,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4203,1,4279.0253,08|03,$GPRMC,131821.0,A,4346.3114,N,2.5422,W,...,A,1100000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
import matplotlib.pyplot as plt
xas= 'Datalogger tijd (s)'
yas= 'Stroom door de motor (A)'
# Maak een plot van kolom 1 (x-as) versus kolom 2 (y-as)
plt.plot(data_file[xas], data_file[yas], linestyle='-', markersize=1)

# Labels voor de assen
plt.xlabel(xas)
plt.ylabel(yas)

# Titel van de plot
plt.title(f'Plot van {xas} vs {yas}')

# Toon de plot
plt.show()

#print(df.iloc[:,1]-df.iloc[0,1])

KeyError: 'Datalogger tijd (s)'